# MCP Benchmark SDK - Complete Usage Guide

This notebook demonstrates **all usage patterns** for the MCP Benchmark SDK, from the simplest harness to custom agents and verifiers.

**Table of Contents:**
1. [Setup](#setup)
2. [Pattern 1: Simplest Harness](#pattern-1-simplest-harness)
3. [Pattern 2: Using Agents Without Harness](#pattern-2-using-agents-without-harness)
4. [Pattern 3: Custom Agent (Qwen Example)](#pattern-3-custom-agent-qwen-example)
5. [Pattern 4: Custom Verifier](#pattern-4-custom-verifier)
6. [Pattern 5: Full Workflow with Harness + Custom Components](#pattern-5-full-workflow-with-harness--custom-components)
7. [Pattern 6: Multiple Models Comparison](#pattern-6-multiple-models-comparison)
8. [Pattern 7: Observers and Progress Tracking](#pattern-7-observers-and-progress-tracking)

**Focus**: The SDK is **harness-first**. The harness orchestrates everything - you define scenarios and let it run.


## Quick Reference

**Minimal harness usage** - The simplest way to get started:

```python
from pathlib import Path
from mcp_benchmark_sdk import TestHarness, TestHarnessConfig, MCPConfig, create_agent

harness = TestHarness(
    harness_path=Path("task.json"),
    config=TestHarnessConfig(
        mcp=MCPConfig(name="jira", url="http://localhost:8015/mcp", transport="streamable_http")
    )
)

results = await harness.run(models=["gpt-4o"], agent_factory=create_agent)
```

**That's it!** The harness handles the rest: agent creation, MCP connections, execution, verification, and metrics.

---


## Setup

First, install the SDK and set up API keys:


In [ ]:
# Install SDK (if not already installed)
# !pip install mcp-benchmark-sdk

import os
import asyncio
import json
from pathlib import Path

# Set up API keys (replace with your actual keys or load from .env)
os.environ["OPENAI_API_KEY"] = "your-openai-key"
os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key"
os.environ["GOOGLE_API_KEY"] = "your-google-key"
# os.environ["DASHSCOPE_API_KEY"] = "your-dashscope-key"  # For Qwen

print("✓ Setup complete!")


✓ Setup complete!


---

## Pattern 1: Simplest Harness

**The recommended way to use the SDK.** Create a harness file and let the SDK handle everything.

### Step 1: Create a Simple Harness File


In [2]:
# Create a simple harness file
simple_harness = {
    "scenarios": [
        {
            "scenario_id": "create_bug",
            "name": "Create Bug Issue",
            "description": "Test if agent can create a bug issue",
            "prompts": [
                {
                    "prompt_text": "Create a bug issue in project DEMO with summary 'Login button not working' and description 'Users cannot click the login button'",
                    "expected_tools": ["create_issue"],
                    "verifier": {
                        "verifier_type": "database_state",
                        "validation_config": {
                            "query": "SELECT COUNT(*) FROM issue WHERE summary = 'Login button not working'",
                            "expected_value": 1,
                            "comparison_type": "equals"
                        }
                    }
                }
            ],
            "metadata": {"difficulty": "easy"},
            "conversation_mode": False
        }
    ]
}

# Save to file
with open("simple_task.json", "w") as f:
    json.dump(simple_harness, f, indent=2)

print("✓ Created simple_task.json")


✓ Created simple_task.json


### Step 2: Run the Harness

This is the **core pattern** - everything else is built on top of this:


In [3]:
from mcp_benchmark_sdk import TestHarness, TestHarnessConfig, MCPConfig, create_agent

async def run_simple_harness():
    # Configure MCP server
    mcp_config = MCPConfig(
        name="jira",
        url="http://localhost:8015/mcp",
        transport="streamable_http"
    )
    
    # Create harness
    harness = TestHarness(
        harness_path=Path("simple_task.json"),
        config=TestHarnessConfig(
            mcp=mcp_config,
            max_steps=50,
            tool_call_limit=100,
            runs_per_scenario=1,
        )
    )
    
    print(f"Loaded {len(harness.scenarios)} scenario(s)")
    
    # Run benchmarks
    results = await harness.run(
        models=["gpt-4o"],
        agent_factory=create_agent,
    )
    
    # Print results
    for result in results:
        status = "✓ PASS" if result.success else "✗ FAIL"
        print(f"\n{result.model} - {result.scenario_id}: {status}")
        print(f"  Steps: {result.result.metadata.get('steps')}")
        print(f"  Database ID: {result.result.database_id}")
        
        if result.error:
            print(f"  Error: {result.error}")
        
        # Show verifier results
        if result.verifier_results:
            print(f"  Verifiers:")
            for v in result.verifier_results:
                v_status = "✓" if v.success else "✗"
                print(f"    {v_status} {v.name}: Expected {v.expected_value}, Got {v.actual_value}")
    
    return results

# Run it
results = await run_simple_harness()
print(f"\n✓ Harness completed! Pass rate: {sum(r.success for r in results)}/{len(results)}")


Loaded 1 scenario(s)

gpt-4o - create_bug: ✗ FAIL
  Steps: 2
  Database ID: 99073ca8-9f71-4204-98a6-0082182611dc
  Error: Verifiers failed: DatabaseVerifier
  Verifiers:
    ✗ DatabaseVerifier: Expected 1, Got 0

✓ Harness completed! Pass rate: 0/1


**That's it!** The harness handled:
- Loading the scenario
- Creating the agent
- Connecting to MCP
- Running the task
- Verifying the result
- Collecting metrics

---


## Pattern 2: Using Agents Without Harness

**Best for:** One-off tasks, interactive testing, custom workflows

You can use agents directly without the harness:


In [4]:
from mcp_benchmark_sdk import ClaudeAgent, Task, MCPConfig

async def run_agent_directly():
    # Create agent
    agent = ClaudeAgent(
        model="claude-sonnet-4-5",
        temperature=0.1,
        tool_call_limit=100,
    )
    
    # Define task
    task = Task(
        prompt="Create a bug issue in project DEMO titled 'Homepage not loading' with description 'Users report 500 error'",
        mcp=MCPConfig(
            name="jira",
            url="http://localhost:8015/mcp",
            transport="streamable_http"
        )
    )
    
    # Run task
    result = await agent.run(task, max_steps=50)
    
    print(f"Success: {result.success}")
    print(f"Steps: {result.metadata.get('steps')}")
    print(f"Database ID: {result.database_id}")
    
    if result.error:
        print(f"Error: {result.error}")
    
    # Access conversation history
    conversation = result.get_conversation_history()
    print(f"\nConversation ({len(conversation)} entries):")
    for i, entry in enumerate(conversation[:5]):  # Show first 5
        if entry["type"] == "message":
            print(f"  {i+1}. {entry['role']}: {entry['content'][:60]}...")
        elif entry["type"] == "tool_call":
            print(f"  {i+1}. Tool: {entry['tool']}")
        elif entry["type"] == "tool_result":
            print(f"  {i+1}. Tool result: {entry['tool']}")
    
    return result

result = await run_agent_directly()
print("\n✓ Direct agent execution complete!")


/var/folders/gw/zj2q1xwn18nd9922p394wrb00000gn/T/ipykernel_59153/2260787270.py:5: UserWarning: Claude thinking mode requires temperature=1.0. Your temperature=0.1 will be overridden. Set temperature=1.0 or enable_thinking=False to suppress this warning.
  agent = ClaudeAgent(


Success: True
Steps: 3
Database ID: 691041b0-252a-43df-aca4-e65c61e0b23a

Conversation (10 entries):
  1. user: Create a bug issue in project DEMO titled 'Homepage not load...
  2. assistant: I'll create a bug issue in the DEMO project with the specifi...
  3. Tool: create_issue
  4. Tool: create_issue
  5. Tool result: create_issue

✓ Direct agent execution complete!


### Manual Verification

When using agents directly, you can manually verify results:


In [ ]:
from mcp_benchmark_sdk import DatabaseVerifier

async def verify_result(result):
    # Create verifier
    verifier = DatabaseVerifier(
        query="SELECT COUNT(*) FROM issue WHERE summary = 'Homepage not loading'",
        expected_value=1,
        mcp_url="http://localhost:8015/mcp",
        database_id=result.database_id,  # Use same database as task
        comparison="equals"
    )
    
    # Run verification
    verifier_result = await verifier.verify()
    
    print(f"Verified: {verifier_result.success}")
    print(f"Expected: {verifier_result.expected_value}, Got: {verifier_result.actual_value}")
    if verifier_result.error:
        print(f"Error: {verifier_result.error}")
    
    return verifier_result

# Verify the previous result
verifier_result = await verify_result(result)
print("\n✓ Verification complete!")


In [ ]:
from mcp_benchmark_sdk import Agent, AgentResponse
from mcp_benchmark_sdk.parsers import OpenAIResponseParser, ResponseParser
from mcp_benchmark_sdk.utils import retry_with_backoff
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, AIMessage

class QwenAgent(Agent):
    """Custom agent for Alibaba Cloud Qwen models.
    
    Uses OpenAI-compatible API from DashScope.
    Requires DASHSCOPE_API_KEY environment variable.
    """
    
    def __init__(
        self,
        model: str = "qwen-plus",
        temperature: float = 0.1,
        max_output_tokens: int | None = None,
        tool_call_limit: int = 1000,
        system_prompt: str | None = None,
    ):
        super().__init__(system_prompt=system_prompt, tool_call_limit=tool_call_limit)
        
        self.model = model
        self.temperature = temperature
        self.max_output_tokens = max_output_tokens
        
        # Model mapping
        model_map = {
            "qwen-14b": "qwen3-14b",
            "qwen-plus": "qwen-plus",
            "qwen-turbo": "qwen-turbo",
            "qwen-max": "qwen-max",
        }
        self.actual_model = model_map.get(model.lower(), model)
        
        # Get base URL and API key
        self.base_url = os.environ.get(
            "DASHSCOPE_BASE_URL",
            "https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
        )
        self.api_key = os.environ.get("DASHSCOPE_API_KEY")
        if not self.api_key:
            print("⚠️  DASHSCOPE_API_KEY not set - agent will fail at runtime")
    
    def _build_llm(self):
        """Build LLM client (called during agent.initialize())."""
        config = {
            "model": self.actual_model,
            "temperature": self.temperature,
            "timeout": None,
            "max_retries": 3,
            "base_url": self.base_url,
            "api_key": self.api_key,
        }
        
        if self.max_output_tokens is not None:
            config["max_completion_tokens"] = self.max_output_tokens
        
        # Disable thinking mode for non-streaming
        config["extra_body"] = {"enable_thinking": False}
        
        llm = ChatOpenAI(**config)
        # Bind tools to LLM (self._tools is set during initialize())
        return llm.bind_tools(self._tools) if self._tools else llm
    
    async def get_response(self, messages: list[BaseMessage]) -> tuple[AgentResponse, AIMessage]:
        """Get model response with retry logic."""
        if not self._llm:
            raise RuntimeError("LLM not initialized. Call initialize() first.")
        
        # Call LLM with retry logic
        async def _invoke():
            return await self._llm.ainvoke(messages)
        
        ai_message = await retry_with_backoff(
            _invoke,
            max_retries=2,
            timeout_seconds=600.0,
            on_retry=lambda attempt, exc, delay: None,
        )
        
        # Parse response using OpenAI parser (compatible format)
        parser = self.get_response_parser()
        parsed = parser.parse(ai_message)
        
        # Convert to AgentResponse
        agent_response = AgentResponse(
            content=parsed.content,
            tool_calls=parsed.tool_calls,
            reasoning="\n".join(parsed.reasoning) if parsed.reasoning else None,
            done=not bool(parsed.tool_calls),
        )
        
        return agent_response, ai_message
    
    def get_response_parser(self) -> ResponseParser:
        """Get OpenAI-compatible response parser."""
        return OpenAIResponseParser()

print("✓ QwenAgent class defined!")


In [ ]:
def custom_agent_factory(model: str, **kwargs):
    """Factory function for creating agents (including custom ones)."""
    if model.startswith("qwen"):
        return QwenAgent(model=model, **kwargs)
    else:
        # Fall back to built-in agents
        return create_agent(model, **kwargs)

async def run_harness_with_custom_agent():
    """Run harness with custom agent factory."""
    harness = TestHarness(
        harness_path=Path("simple_task.json"),
        config=TestHarnessConfig(
            mcp=MCPConfig(
                name="jira",
                url="http://localhost:8015/mcp",
                transport="streamable_http"
            )
        )
    )
    
    # Run with both custom (Qwen) and built-in (GPT) agents
    results = await harness.run(
        models=["gpt-4o"],  # Add "qwen-plus" if you have DASHSCOPE_API_KEY
        agent_factory=custom_agent_factory,
    )
    
    for result in results:
        status = "✓ PASS" if result.success else "✗ FAIL"
        print(f"{result.model}: {status}")
    
    return results

# Run it
# results = await run_harness_with_custom_agent()
print("✓ Custom agent factory ready! Now the harness can use ANY agent you create.")


---

## Pattern 4: Custom Verifier

**Best for:** Complex validation logic, custom result checks

Beyond database queries, you can create verifiers for any validation logic:


In [ ]:
from mcp_benchmark_sdk import Verifier, VerifierResult
import httpx

class APIResponseVerifier(Verifier):
    """Verify that an API endpoint returns expected data.
    
    Example: Check if an issue was created by querying the API directly.
    """
    
    def __init__(
        self,
        endpoint: str,
        expected_field: str,
        expected_value: any,
        name: str | None = None
    ):
        super().__init__(name or "APIResponseVerifier")
        self.endpoint = endpoint
        self.expected_field = expected_field
        self.expected_value = expected_value
    
    async def verify(self) -> VerifierResult:
        """Execute verification."""
        try:
            async with httpx.AsyncClient() as client:
                response = await client.get(self.endpoint)
                response.raise_for_status()
                data = response.json()
                
                actual_value = data.get(self.expected_field)
                success = actual_value == self.expected_value
                
                return VerifierResult(
                    name=self.name,
                    success=success,
                    expected_value=self.expected_value,
                    actual_value=actual_value,
                    comparison_type="equals",
                    error=None if success else "Value mismatch",
                )
        except Exception as exc:
            return VerifierResult(
                name=self.name,
                success=False,
                expected_value=self.expected_value,
                actual_value=None,
                comparison_type="equals",
                error=str(exc),
            )

# Test it
async def test_custom_verifier():
    verifier = APIResponseVerifier(
        endpoint="https://jsonplaceholder.typicode.com/todos/1",
        expected_field="userId",
        expected_value=1,
        name="Check User ID"
    )
    
    result = await verifier.verify()
    print(f"✓ {result.name}: {result.success}")
    print(f"  Expected: {result.expected_value}, Got: {result.actual_value}")

await test_custom_verifier()
print("\n✓ Custom verifier works!")


---

## Pattern 5: Observers for Progress Tracking

**Best for:** Real-time monitoring, debugging, custom logging

Add observers to track execution in real-time:


In [ ]:
from mcp_benchmark_sdk import RunObserver

class DetailedObserver(RunObserver):
    """Observer that tracks everything with nice formatting."""
    
    def __init__(self, label: str):
        self.label = label
        self.stats = {
            "messages": 0,
            "tool_calls": 0,
            "tool_errors": 0,
        }
    
    async def on_message(self, role: str, content: str, metadata=None):
        self.stats["messages"] += 1
        if role == "assistant":
            print(f"[{self.label}] 💬 Agent: {content[:80]}...")
    
    async def on_tool_call(self, tool_name, arguments, result, is_error=False):
        self.stats["tool_calls"] += 1
        if is_error:
            self.stats["tool_errors"] += 1
        status = "✗" if is_error else "✓"
        print(f"[{self.label}] 🔧 Tool {status}: {tool_name}")
    
    async def on_status(self, message: str, level: str = "info"):
        emoji = {"info": "ℹ️", "warning": "⚠️", "error": "❌"}.get(level, "ℹ️")
        print(f"[{self.label}] {emoji} {message}")
    
    def print_stats(self):
        print(f"\n📊 [{self.label}] Statistics:")
        print(f"   Messages: {self.stats['messages']}")
        print(f"   Tool calls: {self.stats['tool_calls']}")
        print(f"   Tool errors: {self.stats['tool_errors']}")

print("✓ DetailedObserver class defined!")


### Use Observer with Harness


In [ ]:
async def run_with_observer():
    """Run harness with detailed progress tracking."""
    
    harness = TestHarness(
        harness_path=Path("simple_task.json"),
        config=TestHarnessConfig(
            mcp=MCPConfig(
                name="jira",
                url="http://localhost:8015/mcp",
                transport="streamable_http"
            )
        )
    )
    
    # Create observer (one per run)
    observer = DetailedObserver("benchmark")
    harness.add_observer_factory(lambda: observer)
    
    print("🚀 Starting benchmark with observer...\n")
    
    # Run
    results = await harness.run(
        models=["gpt-4o"],
        agent_factory=create_agent,
    )
    
    # Print statistics
    observer.print_stats()
    
    # Print results
    print("\n📊 Results:")
    for result in results:
        status = "✓ PASS" if result.success else "✗ FAIL"
        print(f"  {result.scenario_id}: {status}")
    
    return results

# Run with tracking
# results = await run_with_observer()
print("✓ Observer pattern ready!")


---

## Pattern 6: Multiple Models Comparison

**Best for:** Benchmarking across different LLM providers

Compare how different models perform on the same scenarios:


In [ ]:
from collections import defaultdict

async def compare_models():
    """Compare multiple models on the same scenarios."""
    
    harness = TestHarness(
        harness_path=Path("simple_task.json"),
        config=TestHarnessConfig(
            mcp=MCPConfig(
                name="jira",
                url="http://localhost:8015/mcp",
                transport="streamable_http"
            ),
            runs_per_scenario=3,  # Run each 3 times for reliability
            max_concurrent_runs=10,
        )
    )
    
    # Compare multiple models
    models = [
        "gpt-4o",
        "gpt-4o-mini",
        "claude-sonnet-4-5",
        "gemini-2.0-flash-exp",
    ]
    
    print(f"🏁 Comparing {len(models)} models...\n")
    
    results = await harness.run(
        models=models,
        agent_factory=create_agent,
    )
    
    # Aggregate by model
    model_stats = defaultdict(lambda: {"total": 0, "passed": 0, "steps": []})
    
    for result in results:
        model_stats[result.model]["total"] += 1
        if result.success:
            model_stats[result.model]["passed"] += 1
        model_stats[result.model]["steps"].append(result.result.metadata.get("steps", 0))
    
    # Print comparison
    print("\n" + "="*70)
    print("📊 MODEL COMPARISON")
    print("="*70)
    print(f"{'Model':<30} {'Pass Rate':<15} {'Avg Steps':<10}")
    print("-"*70)
    
    for model, stats in sorted(model_stats.items()):
        pass_rate = stats["passed"] / stats["total"] * 100
        avg_steps = sum(stats["steps"]) / len(stats["steps"]) if stats["steps"] else 0
        print(f"{model:<30} {pass_rate:>6.1f}%          {avg_steps:>6.1f}")
    
    print("="*70)
    
    return results, model_stats

# Run comparison
# results, stats = await compare_models()
print("✓ Model comparison ready!")


---

## Pattern 7: Export Results

**Best for:** Analysis, reporting, debugging

Export results to JSON for further analysis:


In [ ]:
def export_results(results, filename="results.json"):
    """Export results to JSON file with full details."""
    data = [r.to_dict() for r in results]
    
    with open(filename, "w") as f:
        json.dump(data, f, indent=2)
    
    print(f"✓ Exported {len(results)} results to {filename}")
    
    # Show what's included
    if results:
        sample = results[0].to_dict()
        print(f"\n📦 Each result includes:")
        print(f"   - model, scenario_id, scenario_name")
        print(f"   - success, error")
        print(f"   - steps, database_id")
        print(f"   - conversation ({len(sample.get('conversation', []))} entries)")
        print(f"   - verifier_results")
        print(f"   - reasoning_traces (if available)")
        
        # Show conversation structure
        if sample.get('conversation'):
            print(f"\n💬 Conversation format:")
            entry = sample['conversation'][0]
            print(f"   Type: {entry.get('type')}")
            print(f"   Keys: {list(entry.keys())}")

# Example usage:
# async def save_results():
#     results = await run_simple_harness()
#     export_results(results, "benchmark_results.json")
#     return results

print("✓ Export function ready!")


---

## Summary

**🎉 You've learned all the core patterns!**

### The SDK Philosophy: **Harness-First**

The TestHarness is the main component. It orchestrates:
- ✅ Agent creation
- ✅ MCP connections
- ✅ Task execution
- ✅ Result verification
- ✅ Metrics collection

### Usage Patterns Recap

1. **Simplest Harness** ⭐ - Start here!
   - Create JSON file with scenarios
   - Run with `harness.run(models, agent_factory)`
   - Get results with verification

2. **Direct Agent Usage** - For one-off tasks
   - `agent.run(task)` without harness
   - Manual verification with `DatabaseVerifier`

3. **Custom Agents** - Integrate any LLM
   - Subclass `Agent`
   - Implement `_build_llm()`, `get_response()`, `get_response_parser()`
   - Use with harness via custom factory

4. **Custom Verifiers** - Complex validation
   - Subclass `Verifier`
   - Implement `verify()` method
   - Use programmatically (harness integration requires extending loader)

5. **Observers** - Real-time tracking
   - Subclass `RunObserver`
   - Track messages, tool calls, status updates
   - Add to harness with `add_observer_factory()`

6. **Model Comparison** - Systematic benchmarking
   - Pass multiple models to `harness.run()`
   - Aggregate and compare results
   - Statistical analysis (multiple runs per scenario)

7. **Export Results** - Analysis and reporting
   - `result.to_dict()` for JSON serialization
   - Includes conversation, verifiers, reasoning traces
   - Ready for pandas, matplotlib, etc.

### Key Takeaways

✅ **Start with the harness** - It's the recommended approach  
✅ **The harness orchestrates everything** - Agents, MCP, verification  
✅ **Custom agents integrate seamlessly** - Just implement 3 methods  
✅ **Observers provide visibility** - Real-time progress tracking  
✅ **Results are export-ready** - JSON with full details  

### Next Steps

1. Read the full README for API reference
2. Create your own harness files (see `9_tasks/task1.json` for examples)
3. Build custom agents for your LLM providers
4. Create custom verifiers for your use cases
5. Run large-scale benchmarks!

---

## Additional Resources

- **README.md** - Complete API reference
- **simple_harness_example.py** - Basic example script
- **9_tasks/** - Real benchmark scenarios
- **QwenAgent** - Example custom agent implementation

Happy benchmarking! 🎯
